# Potato Disease Classification

## Importing Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt

## Data Information

In [12]:
IMAGE_SIZE = (256,256)
BATCH_SIZE = 32
CHANNELS = 3

## Data Augmentation

In [ ]:
train_generator = image.ImageDataGenerator(
    rotation_range = 15,
    vertical_flip = True,
    horizontal_flip = True,
    rescale = 1./255,
)
validation_generator = image.ImageDataGenerator(
    rotation_range = 15,
    vertical_flip = True,
    horizontal_flip = True,
    rescale = 1./255,
)
test_generator = image.ImageDataGenerator(
    rotation_range = 15,
    vertical_flip = True,
    horizontal_flip = True,
    rescale = 1./255,
)

## Splitting data

In [ ]:
train_dataset = train_generator.flow_from_directory(
    '/content/Train',
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    shuffle = True,
    class_mode = 'categorical'
)
validation_dataset = train_generator.flow_from_directory(
    '/content/Valid',
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    shuffle = True,
    class_mode = 'categorical'
)
test_dataset = train_generator.flow_from_directory(
    '/content/Test',
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    shuffle = True,
    class_mode = 'categorical'
)
class_names = train_dataset.class_indices
n_classes = len(class_names)

## Building Convolutional Neural Network

In [ ]:
vgg16_model = VGG16(weights='imagenet',include_top=False, input_shape=(256,256,3))
model = models.Sequential([
    vgg16_model,
    layers.Flatten(),
    layers.Dense(256,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(n_classes,activation='softmax')
])


In [ ]:
model.summary()

### Compling Neural Model

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### Early Stopping

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=0,
    patience = 3,
)

In [ ]:
history = model.fit(
    train_dataset,
    epochs = 50,
    verbose = 1,
    validation_data = validation_dataset,
    callbacks = [early_stopping]
)

In [ ]:
scores = model.evaluate(test_dataset)
print(scores)

In [ ]:
key_dict = history.history

In [ ]:
loss = key_dict['loss']
val_loss = key_dict['val_loss']

accuracy = key_dict['accuracy']
val_accuracy = key_dict['val_accuracy']

## Visualizing Accuracy and Loss

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(loss,label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

plt.subplot(1,2,2)
plt.plot(accuracy,label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

## Saving model

In [ ]:
model.save('Trained Models\potato_model_ver_1.0.h5')